In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge

In [2]:
merged_data_train = pd.read_csv("simple_train.csv")
merged_data_test = pd.read_csv("simple_test.csv")

In [3]:
merged_data_train.shape,merged_data_test.shape

((721081, 23), (658493, 22))

In [4]:
X = merged_data_train.drop(columns=['Target'])
y = merged_data_train['Target']

In [5]:
cat_columns = X.select_dtypes(include=['object']).columns
cat_features_idx = [X.columns.get_loc(col) for col in cat_columns]

In [6]:
cat_features_idx

[]

In [7]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42,
                                                   )

# Define model and its hyperparameters
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1, 
                           depth=6, 
                           l2_leaf_reg=3, 
                           loss_function='Logloss', 
                           random_seed=42,
                          cat_features=cat_features_idx)

# Train model on training data
model.fit(X_train, y_train, 
          eval_set=(X_test, y_test), 
          verbose=100, 
          early_stopping_rounds=50)



0:	learn: 0.4037672	test: 0.4037860	best: 0.4037860 (0)	total: 371ms	remaining: 6m 10s
100:	learn: 0.0052696	test: 0.0057232	best: 0.0057232 (100)	total: 8.82s	remaining: 1m 18s
200:	learn: 0.0042786	test: 0.0051478	best: 0.0051478 (200)	total: 16.4s	remaining: 1m 5s
300:	learn: 0.0036445	test: 0.0047731	best: 0.0047731 (300)	total: 23.7s	remaining: 55.1s
400:	learn: 0.0031921	test: 0.0046096	best: 0.0046088 (399)	total: 30.8s	remaining: 46.1s
500:	learn: 0.0028476	test: 0.0044608	best: 0.0044608 (500)	total: 38s	remaining: 37.8s
600:	learn: 0.0025632	test: 0.0043748	best: 0.0043734 (595)	total: 45.3s	remaining: 30.1s
700:	learn: 0.0023239	test: 0.0042955	best: 0.0042955 (700)	total: 53.2s	remaining: 22.7s
800:	learn: 0.0021488	test: 0.0042799	best: 0.0042799 (776)	total: 1m	remaining: 15.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.004279870786
bestIteration = 776

Shrink model to first 777 iterations.


In [8]:
# Evaluate model on testing data
score = model.score(X_test, y_test)
print(f"Accuracy on testing data: {score}")

Accuracy on testing data: 0.9989737686957848


In [9]:
y_pred = model.predict(X_test)

In [10]:
from sklearn.metrics import classification_report, precision_recall_curve, auc

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    138842
           1       1.00      0.97      0.99      5375

    accuracy                           1.00    144217
   macro avg       1.00      0.99      0.99    144217
weighted avg       1.00      1.00      1.00    144217



In [12]:
precision, recall, thrs = precision_recall_curve(y_test, y_pred)

In [13]:
max(0, 100 * auc(precision, recall))

94.94180861612732

In [14]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from skopt import BayesSearchCV

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CatBoost model for classification
model = CatBoostClassifier()

# Define the hyperparameter search space for Bayesian optimization
param_space = {
    'max_depth': (3, 7),
    'learning_rate': (0.01, 0.3, 'log-uniform'),
    'n_estimators': (100, 300),
}

# Perform Bayesian optimization to find the best hyperparameters
bayes_search = BayesSearchCV(estimator=model, search_spaces=param_space, scoring='f1_macro', cv=5)
bayes_search.fit(X_train, y_train)

# Get the best model from the Bayesian optimization
best_model = bayes_search.best_estimator_

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Compute the F1 score
f1 = f1_score(y_test, y_pred)

# Print the classification report
report = classification_report(y_test, y_pred)
print("F1 Score:", f1)
print("Classification Report:\n", report)


0:	learn: 0.6165890	total: 138ms	remaining: 26.5s
1:	learn: 0.5475159	total: 272ms	remaining: 26s
2:	learn: 0.4852018	total: 342ms	remaining: 21.7s
3:	learn: 0.4321300	total: 412ms	remaining: 19.5s
4:	learn: 0.3822113	total: 473ms	remaining: 17.8s
5:	learn: 0.3480942	total: 523ms	remaining: 16.3s
6:	learn: 0.3095716	total: 584ms	remaining: 15.5s
7:	learn: 0.2797650	total: 637ms	remaining: 14.7s
8:	learn: 0.2533520	total: 690ms	remaining: 14.1s
9:	learn: 0.2264220	total: 745ms	remaining: 13.6s
10:	learn: 0.2030860	total: 824ms	remaining: 13.6s
11:	learn: 0.1853994	total: 878ms	remaining: 13.2s
12:	learn: 0.1716871	total: 935ms	remaining: 12.9s
13:	learn: 0.1592360	total: 988ms	remaining: 12.6s
14:	learn: 0.1441889	total: 1.05s	remaining: 12.5s
15:	learn: 0.1344260	total: 1.1s	remaining: 12.2s
16:	learn: 0.1219585	total: 1.15s	remaining: 11.9s
17:	learn: 0.1113037	total: 1.2s	remaining: 11.7s
18:	learn: 0.1045661	total: 1.26s	remaining: 11.5s
19:	learn: 0.0984113	total: 1.32s	remaining: 

164:	learn: 0.0095157	total: 10.4s	remaining: 1.76s
165:	learn: 0.0094895	total: 10.4s	remaining: 1.7s
166:	learn: 0.0094689	total: 10.5s	remaining: 1.64s
167:	learn: 0.0094466	total: 10.6s	remaining: 1.58s
168:	learn: 0.0094196	total: 10.7s	remaining: 1.52s
169:	learn: 0.0094017	total: 10.7s	remaining: 1.45s
170:	learn: 0.0093807	total: 10.8s	remaining: 1.39s
171:	learn: 0.0093629	total: 10.9s	remaining: 1.33s
172:	learn: 0.0093419	total: 11s	remaining: 1.27s
173:	learn: 0.0093225	total: 11s	remaining: 1.2s
174:	learn: 0.0093065	total: 11.1s	remaining: 1.14s
175:	learn: 0.0092718	total: 11.1s	remaining: 1.08s
176:	learn: 0.0092519	total: 11.2s	remaining: 1.01s
177:	learn: 0.0092168	total: 11.3s	remaining: 950ms
178:	learn: 0.0091916	total: 11.3s	remaining: 886ms
179:	learn: 0.0091739	total: 11.4s	remaining: 822ms
180:	learn: 0.0091479	total: 11.4s	remaining: 758ms
181:	learn: 0.0091294	total: 11.5s	remaining: 695ms
182:	learn: 0.0091043	total: 11.5s	remaining: 631ms
183:	learn: 0.0090

133:	learn: 0.0109211	total: 8.88s	remaining: 3.91s
134:	learn: 0.0108795	total: 8.96s	remaining: 3.85s
135:	learn: 0.0108438	total: 9.03s	remaining: 3.78s
136:	learn: 0.0107633	total: 9.11s	remaining: 3.72s
137:	learn: 0.0107243	total: 9.18s	remaining: 3.66s
138:	learn: 0.0106897	total: 9.25s	remaining: 3.6s
139:	learn: 0.0106380	total: 9.33s	remaining: 3.53s
140:	learn: 0.0106012	total: 9.4s	remaining: 3.47s
141:	learn: 0.0105582	total: 9.47s	remaining: 3.4s
142:	learn: 0.0105179	total: 9.55s	remaining: 3.34s
143:	learn: 0.0104840	total: 9.61s	remaining: 3.27s
144:	learn: 0.0103508	total: 9.69s	remaining: 3.21s
145:	learn: 0.0103034	total: 9.76s	remaining: 3.14s
146:	learn: 0.0102715	total: 9.83s	remaining: 3.08s
147:	learn: 0.0102195	total: 9.91s	remaining: 3.01s
148:	learn: 0.0101893	total: 10.1s	remaining: 2.98s
149:	learn: 0.0101583	total: 10.2s	remaining: 2.91s
150:	learn: 0.0101310	total: 10.2s	remaining: 2.85s
151:	learn: 0.0101067	total: 10.3s	remaining: 2.78s
152:	learn: 0.0

103:	learn: 0.0120783	total: 7.81s	remaining: 6.68s
104:	learn: 0.0120168	total: 7.88s	remaining: 6.61s
105:	learn: 0.0119476	total: 7.95s	remaining: 6.53s
106:	learn: 0.0118875	total: 8.03s	remaining: 6.45s
107:	learn: 0.0118318	total: 8.1s	remaining: 6.37s
108:	learn: 0.0117634	total: 8.16s	remaining: 6.29s
109:	learn: 0.0117037	total: 8.24s	remaining: 6.21s
110:	learn: 0.0116301	total: 8.31s	remaining: 6.14s
111:	learn: 0.0115310	total: 8.38s	remaining: 6.06s
112:	learn: 0.0114735	total: 8.46s	remaining: 5.99s
113:	learn: 0.0114189	total: 8.53s	remaining: 5.91s
114:	learn: 0.0113703	total: 8.61s	remaining: 5.84s
115:	learn: 0.0112923	total: 8.68s	remaining: 5.76s
116:	learn: 0.0112308	total: 8.75s	remaining: 5.68s
117:	learn: 0.0111853	total: 8.82s	remaining: 5.61s
118:	learn: 0.0111316	total: 8.89s	remaining: 5.53s
119:	learn: 0.0110833	total: 8.97s	remaining: 5.46s
120:	learn: 0.0110456	total: 9.04s	remaining: 5.38s
121:	learn: 0.0109990	total: 9.11s	remaining: 5.3s
122:	learn: 0.

71:	learn: 0.0144331	total: 5.35s	remaining: 8.98s
72:	learn: 0.0142674	total: 5.43s	remaining: 8.92s
73:	learn: 0.0141723	total: 5.5s	remaining: 8.85s
74:	learn: 0.0140054	total: 5.58s	remaining: 8.78s
75:	learn: 0.0138461	total: 5.66s	remaining: 8.71s
76:	learn: 0.0137288	total: 5.73s	remaining: 8.63s
77:	learn: 0.0136118	total: 5.81s	remaining: 8.57s
78:	learn: 0.0135015	total: 5.89s	remaining: 8.49s
79:	learn: 0.0134011	total: 5.96s	remaining: 8.42s
80:	learn: 0.0133285	total: 6.04s	remaining: 8.35s
81:	learn: 0.0132375	total: 6.12s	remaining: 8.28s
82:	learn: 0.0131524	total: 6.19s	remaining: 8.2s
83:	learn: 0.0130548	total: 6.27s	remaining: 8.13s
84:	learn: 0.0130095	total: 6.34s	remaining: 8.05s
85:	learn: 0.0128899	total: 6.42s	remaining: 7.98s
86:	learn: 0.0127606	total: 6.49s	remaining: 7.91s
87:	learn: 0.0126901	total: 6.55s	remaining: 7.82s
88:	learn: 0.0125633	total: 6.63s	remaining: 7.74s
89:	learn: 0.0124622	total: 6.69s	remaining: 7.66s
90:	learn: 0.0123528	total: 6.77s

...

195:	learn: 0.0142383	total: 11.4s	remaining: 5.81s
196:	learn: 0.0142031	total: 11.5s	remaining: 5.76s
197:	learn: 0.0141422	total: 11.5s	remaining: 5.7s
198:	learn: 0.0140897	total: 11.6s	remaining: 5.64s
199:	learn: 0.0140244	total: 11.6s	remaining: 5.58s
200:	learn: 0.0139618	total: 11.7s	remaining: 5.52s
201:	learn: 0.0139035	total: 11.7s	remaining: 5.46s
202:	learn: 0.0138553	total: 11.8s	remaining: 5.41s
203:	learn: 0.0137993	total: 11.9s	remaining: 5.35s
204:	learn: 0.0137250	total: 11.9s	remaining: 5.29s
205:	learn: 0.0136728	total: 12s	remaining: 5.23s
206:	learn: 0.0136183	total: 12s	remaining: 5.17s
207:	learn: 0.0135772	total: 12.1s	remaining: 5.11s
208:	learn: 0.0135498	total: 12.1s	remaining: 5.06s
209:	learn: 0.0135122	total: 12.2s	remaining: 5s
210:	learn: 0.0134526	total: 12.3s	remaining: 4.94s
211:	learn: 0.0134046	total: 12.3s	remaining: 4.88s
212:	learn: 0.0133542	total: 12.4s	remaining: 4.82s
213:	learn: 0.0133023	total: 12.4s	remaining: 4.76s
214:	learn: 0.013259

60:	learn: 0.0614400	total: 3.42s	remaining: 13.2s
61:	learn: 0.0595473	total: 3.48s	remaining: 13.2s
62:	learn: 0.0577871	total: 3.54s	remaining: 13.1s
63:	learn: 0.0560685	total: 3.6s	remaining: 13s
64:	learn: 0.0544353	total: 3.65s	remaining: 13s
65:	learn: 0.0528781	total: 3.71s	remaining: 12.9s
66:	learn: 0.0516745	total: 3.77s	remaining: 12.9s
67:	learn: 0.0502560	total: 3.82s	remaining: 12.8s
68:	learn: 0.0492182	total: 3.88s	remaining: 12.8s
69:	learn: 0.0479265	total: 3.94s	remaining: 12.7s
70:	learn: 0.0466371	total: 3.99s	remaining: 12.7s
71:	learn: 0.0455884	total: 4.05s	remaining: 12.6s
72:	learn: 0.0444266	total: 4.1s	remaining: 12.5s
73:	learn: 0.0433593	total: 4.17s	remaining: 12.5s
74:	learn: 0.0423382	total: 4.22s	remaining: 12.4s
75:	learn: 0.0414675	total: 4.28s	remaining: 12.4s
76:	learn: 0.0405902	total: 4.34s	remaining: 12.3s
77:	learn: 0.0396857	total: 4.4s	remaining: 12.3s
78:	learn: 0.0388260	total: 4.46s	remaining: 12.2s
79:	learn: 0.0381187	total: 4.51s	rema

220:	learn: 0.0128180	total: 12.8s	remaining: 4.36s
221:	learn: 0.0127767	total: 12.9s	remaining: 4.3s
222:	learn: 0.0127535	total: 13s	remaining: 4.24s
223:	learn: 0.0127190	total: 13s	remaining: 4.18s
224:	learn: 0.0126828	total: 13.1s	remaining: 4.13s
225:	learn: 0.0126521	total: 13.1s	remaining: 4.07s
226:	learn: 0.0126114	total: 13.2s	remaining: 4.01s
227:	learn: 0.0125823	total: 13.2s	remaining: 3.95s
228:	learn: 0.0125396	total: 13.3s	remaining: 3.89s
229:	learn: 0.0125076	total: 13.4s	remaining: 3.83s
230:	learn: 0.0124728	total: 13.4s	remaining: 3.77s
231:	learn: 0.0124379	total: 13.5s	remaining: 3.71s
232:	learn: 0.0124013	total: 13.5s	remaining: 3.66s
233:	learn: 0.0123637	total: 13.6s	remaining: 3.6s
234:	learn: 0.0123201	total: 13.6s	remaining: 3.54s
235:	learn: 0.0122884	total: 13.7s	remaining: 3.48s
236:	learn: 0.0122560	total: 13.8s	remaining: 3.43s
237:	learn: 0.0122233	total: 13.8s	remaining: 3.37s
238:	learn: 0.0121931	total: 13.9s	remaining: 3.31s
239:	learn: 0.0121

87:	learn: 0.0031790	total: 7.17s	remaining: 17.3s
88:	learn: 0.0031711	total: 7.25s	remaining: 17.2s
89:	learn: 0.0031566	total: 7.33s	remaining: 17.1s
90:	learn: 0.0031385	total: 7.42s	remaining: 17.1s
91:	learn: 0.0031191	total: 7.51s	remaining: 17s
92:	learn: 0.0030904	total: 7.58s	remaining: 16.9s
93:	learn: 0.0030754	total: 7.68s	remaining: 16.8s
94:	learn: 0.0030754	total: 7.73s	remaining: 16.7s
95:	learn: 0.0030694	total: 7.81s	remaining: 16.6s
96:	learn: 0.0030276	total: 7.89s	remaining: 16.5s
97:	learn: 0.0030052	total: 7.97s	remaining: 16.4s
98:	learn: 0.0029876	total: 8.05s	remaining: 16.4s
99:	learn: 0.0029751	total: 8.14s	remaining: 16.3s
100:	learn: 0.0029668	total: 8.23s	remaining: 16.2s
101:	learn: 0.0029515	total: 8.31s	remaining: 16.1s
102:	learn: 0.0029337	total: 8.41s	remaining: 16.1s
103:	learn: 0.0029199	total: 8.49s	remaining: 16s
104:	learn: 0.0029116	total: 8.59s	remaining: 15.9s
105:	learn: 0.0028954	total: 8.69s	remaining: 15.9s
106:	learn: 0.0028734	total: 

247:	learn: 0.0016068	total: 20.3s	remaining: 4.26s
248:	learn: 0.0015940	total: 20.4s	remaining: 4.18s
249:	learn: 0.0015860	total: 20.5s	remaining: 4.1s
250:	learn: 0.0015858	total: 20.6s	remaining: 4.02s
251:	learn: 0.0015841	total: 20.7s	remaining: 3.93s
252:	learn: 0.0015779	total: 20.7s	remaining: 3.85s
253:	learn: 0.0015709	total: 20.8s	remaining: 3.77s
254:	learn: 0.0015642	total: 20.9s	remaining: 3.69s
255:	learn: 0.0015641	total: 21s	remaining: 3.6s
256:	learn: 0.0015580	total: 21s	remaining: 3.52s
257:	learn: 0.0015548	total: 21.1s	remaining: 3.44s
258:	learn: 0.0015474	total: 21.2s	remaining: 3.35s
259:	learn: 0.0015447	total: 21.3s	remaining: 3.27s
260:	learn: 0.0015428	total: 21.4s	remaining: 3.19s
261:	learn: 0.0015428	total: 21.4s	remaining: 3.11s
262:	learn: 0.0015428	total: 21.5s	remaining: 3.02s
263:	learn: 0.0015428	total: 21.6s	remaining: 2.94s
264:	learn: 0.0015418	total: 21.6s	remaining: 2.85s
265:	learn: 0.0015345	total: 21.7s	remaining: 2.77s
266:	learn: 0.0015

In [15]:
precision, recall, thrs = precision_recall_curve(y_test, y_pred)

In [16]:
max(0, 100 * auc(precision, recall))

94.93726494949466

In [17]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost model for classification
model = xgb.XGBClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Compute the F1 score
f1 = f1_score(y_test, y_pred)

# Print the classification report
report = classification_report(y_test, y_pred)
print("F1 Score:", f1)
print("Classification Report:\n", report)


F1 Score: 0.9849340866290018
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    138842
           1       1.00      0.97      0.98      5375

    accuracy                           1.00    144217
   macro avg       1.00      0.99      0.99    144217
weighted avg       1.00      1.00      1.00    144217



In [18]:
precision, recall, thrs = precision_recall_curve(y_test, y_pred)

In [19]:
max(0, 100 * auc(precision, recall))

94.83141845655678

In [20]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost model for classification
model = xgb.XGBClassifier()

# Define the hyperparameter search space for Bayesian optimization
param_space = {
    'max_depth': (3, 7),
    'learning_rate': (0.001, 0.1, 'log-uniform'),
    'n_estimators': (100, 300),
}

# Perform Bayesian optimization to find the best hyperparameters
bayes_search = BayesSearchCV(estimator=model, search_spaces=param_space, scoring='f1_macro', cv=5)
bayes_search.fit(X_train, y_train)

# Get the best model from the Bayesian optimization
best_model = bayes_search.best_estimator_

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Compute the F1 score
f1 = f1_score(y_test, y_pred)

# Print the classification report
report = classification_report(y_test, y_pred)
print("F1 Score:", f1)
print("Classification Report:\n", report)


C:\Users\rahul\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\rahul\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\rahul\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\rahul\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\rahul\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\rahul\

F1 Score: 0.9856873822975519
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    138842
           1       1.00      0.97      0.99      5375

    accuracy                           1.00    144217
   macro avg       1.00      0.99      0.99    144217
weighted avg       1.00      1.00      1.00    144217



In [21]:
precision, recall, thrs = precision_recall_curve(y_test, y_pred)

In [22]:
max(0, 100 * auc(precision, recall))

94.90537251368345